In [2]:
import cv2
import os
import mediapipe as mp

In [3]:
def clear_directory(directory):
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for filename in os.listdir(label_dir):
            file_path = os.path.join(label_dir, filename)
            # print(filename)
            os.remove(file_path)

In [4]:
clear_directory('../project_code/datasets/')

In [5]:
def detectHand(image):
    # Khởi tạo đối tượng Hands trong MediaPipe
    mp_hands = mp.solutions.hands.Hands()
    handlm = []
    # Chuyển đổi ảnh từ BGR sang RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Đưa ảnh vào MediaPipe Hands để nhận dạng bàn tay
    results = mp_hands.process(image_rgb)

    # Vẽ các điểm keypoint trên ảnh
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            
            for landmark in hand_landmarks.landmark:
                # Lấy tọa độ của các điểm keypoint
                px = int(landmark.x * image.shape[1])
                py = int(landmark.y * image.shape[0])
                pz = int(landmark.z * image.shape[2])
                handlm.append([px, py, pz])
                
                # Vẽ điểm keypoint
                cv2.circle(image, (px, py), 5, (0, 255, 0), -1)
                
        mp.solutions.drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
        
    return  image

In [6]:


# Kết nối với camera
cap = cv2.VideoCapture(0)
# width_new = 1280
# height_new = 720

# # Tạo cửa sổ imshow với kích thước tùy chỉnh


width, height = 1280,720

# Vị trí và kích thước của khung chứa dữ liệu
x, y, w, h = 100, 100, 400, 400

# Biến đếm số khung hình đã lưu

num_train = 200
Start = False
# Tạo thư mục lưu trữ dataset (nếu chưa tồn tại)
dataset_dir = '../project_code/datasets/'
rock_dir = os.path.join(dataset_dir, 'rock')
paper_dir = os.path.join(dataset_dir, 'paper')
scissors_dir = os.path.join(dataset_dir, 'scissors')
frame_count = 0

# Dòng chú thích
text = 'r is rock, p is paper, s is scissors'

while True:
    # Đọc khung hình từ camera
    


    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (width, height))
    
    if frame_count == num_train:
        Start = False
        frame_count = 0
    # Vẽ khung chứa dữ liệu
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(frame, "Collecting {}".format(frame_count),
            (80, 80), cv2.FONT_HERSHEY_SIMPLEX , 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    # Hiển thị dòng chú thích
    cv2.putText(frame, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Hiển thị khung hình
    

    # Kiểm tra phím nhấn
    key = cv2.waitKey(10) & 0xFF
    if key == ord('r'):  # Phím 'r' - Rock
        label_dir = rock_dir
        Start = not Start
    elif key == ord('p'):  # Phím 'p' - Paper
        label_dir = paper_dir
        Start = not Start
    elif key == ord('s'):  # Phím 's' - Scissors
        label_dir = scissors_dir
        Start = not Start
    elif key == ord('q'):  # Phím 'q' - Thoát
        break
    

    if Start : 
        

        roi = frame[y:y + h, x:x + w]  # Lấy vùng chứa dữ liệu
        
        # Đường dẫn lưu trữ
        file_path = os.path.join(label_dir, f'frame_{frame_count +1}.jpg')
        
        # Lưu khung hình vào thư mục dataset
        cv2.imwrite(file_path, roi)
            
        frame_count += 1
    cv2.imshow('Camera', frame)

cap.release()
cv2.destroyAllWindows()
